In [26]:
import pandas as pd
from pathlib import Path

RAW = Path("../data/raw")        # where your CSVs are
PROC = Path("../data/processed") # where we'll save outputs
PROC.mkdir(parents=True, exist_ok=True)

charts_path = RAW / "Hot Stuff.csv"
feats_path  = RAW / "Hot 100 Audio Features.csv"

In [27]:
charts = pd.read_csv(charts_path)
feats  = pd.read_csv(feats_path)

In [28]:
## Data Cleaning

As the datasets have identical matches for Song and Performer, I merge the charts and audio features files into one dataframe.

For column names that have identical matches, I append _chart or _feat to specify which data it is using (e.g. index_feat & index_chart).

In [29]:
merged = charts.merge(
    feats,
    on=["Song", "Performer"],  # columns that must match in both files
    how="inner",               # keep only rows that match on BOTH sides
    suffixes=("_chart", "_feat")
)

print("Merged shape:", merged.shape)

Merged shape: (330217, 32)


,index_chart,url,WeekID,Week Position,Song,Performer,SongID_chart,Instance,Previous Week Position,Peak Position,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,0,http://www.billboard.com/charts/hot-100/1965-0...,7/17/1965,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,...,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0
1,1,http://www.billboard.com/charts/hot-100/1965-0...,7/24/1965,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,...,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0
2,2,http://www.billboard.com/charts/hot-100/1965-0...,7/31/1965,14,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,22.0,14,...,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0


Remove any songs that are missing values within the audio feature columns.

In [30]:
feature_cols = [
    "danceability", "energy", "valence", "tempo", "loudness",
    "acousticness", "instrumentalness", "liveness", "speechiness"
]

before = len(merged)
merged = merged.dropna(subset=feature_cols)
after = len(merged)

print(f"Dropped {before - after} rows with missing feature values. Remaining: {after}")

Dropped 43709 rows with missing feature values. Remaining: 286508


Since this dataset tracks the Billboard Hot 100 per week, many songs appear several times within the chart. For the sake of simplicity and a broader view of the data, I will only be using the Peak Position of each song. This means all other instances less than the Peak Position will be deleted.

This results in sparser data per year but information like longevity is still kept in the "Weeks on Chart" column. 

In [31]:
merged_sorted = merged.sort_values("Week Position", ascending=True)
len(merged_sorted)

286508

Delete duplicates that have the same song name and artist, only keeping the first instance (the highest position based off our previous line of code)

In [32]:
peak_songs = (
    merged_sorted
    .drop_duplicates(subset=["Song", "Performer"], keep="first")
    .copy()
)

print("Unique songs kept:", len(peak_songs))

Unique songs kept: 24221


Keep only the songs between the Billboard position of 1-50, effectively cutting the data set in half.

In [33]:
df = peak_songs.loc[peak_songs["Week Position"] <= 50].copy()
print("Songs that peaked in Top-50:", len(df))

Songs that peaked in Top-50: 13732


Convert the WeekID into a datetime format which lets us sort the data set properly.

In [34]:
df["chart_week"] = pd.to_datetime(df["WeekID"], errors="coerce")

Create two new columns, year and decade to help us aggregate the data in a simple manner.

In [35]:
df['year'] = pd.to_datetime(df['WeekID']).dt.year
df['decade'] = (df['year'] // 10) * 10

Clean up the data by dropping irrelevant columns, renaming certain ones, and keeping consistency among column names.

In [37]:
cols_to_drop = [
    "url", "Instance", "Previous Week Position", "Peak Position",
    "index_feat", "SongID_chart", "SongID_feat",
    "spotify_track_id", "spotify_track_preview_url",
    "spotify_track_album", "WeekID", "liveness", "spotify_track_explicit", "spotify_track_popularity", "key"
]

df = df.drop(columns=cols_to_drop, errors="ignore")

rename_map = {
    "index_chart": "index",
    "chart_week": "weekID",
    "spotify_genre": "genre",
    "spotify_track_duration_ms": "duration"
}

df = df.rename(columns=rename_map)

df.columns = [c.lower() for c in df.columns]

df.columns.tolist()

['index',
 'week position',
 'song',
 'performer',
 'weeks on chart',
 'genre',
 'duration',
 'danceability',
 'energy',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'valence',
 'tempo',
 'time_signature',
 'weekid',
 'year',
 'decade']

Convert the duration from milliseconds into minutes.

In [38]:
df["duration"] = (df["duration"] / (1000 * 60)).round(2)

Move weekid to the 2nd column for legibility.

In [39]:
cols = df.columns.tolist()  # get current column order
cols.remove("weekid")                # temporarily remove it
cols.insert(1, "weekid")             # re-insert at position 1 (2nd place)
df = df[cols]        # reorder DataFrame

Sort the dataframe from oldest to newest week. Within each week, sort from the highest rank (1) to the lowest.

In [40]:
df = df.sort_values(
    by=["weekid", "week position"],
    ascending=[True, True]
).reset_index(drop=True)

In [41]:
df.head(3)

,index,weekid,week position,song,performer,weeks on chart,genre,duration,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,time_signature,year,decade
0,964,1958-08-02,2,Patricia,Perez Prado And His Orchestra,1,"['mambo', 'space age pop']",2.33,0.699,0.715,-5.976,1.0,0.0391,0.180,0.415000,0.810,137.373,4.0,1958,1950
1,980,1958-08-02,3,Splish Splash,Bobby Darin,1,"['adult standards', 'brill building pop', 'eas...",2.20,0.645,0.943,-1.526,1.0,0.0393,0.385,0.000000,0.965,147.768,4.0,1958,1950
2,292513,1958-08-02,7,Yakety Yak,The Coasters,1,"['brill building pop', 'bubblegum pop', 'doo-w...",1.88,0.715,0.669,-9.491,1.0,0.1280,0.705,0.000732,0.976,120.789,4.0,1958,1950


In [42]:
PROC = Path("../data/processed")
PROC.mkdir(parents=True, exist_ok=True)

clean_path = PROC / "merged_df_clean.csv"
df.to_csv(clean_path, index=False)
print("Saved cleaned dataset to:", clean_path)

Saved cleaned dataset to: ../data/processed/merged_df_clean.csv
